# Bias-variance

---

In this part you will explore how bias and variance change using a dataset on college statistics.

In [35]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import patsy

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.model_selection import cross_val_score, cross_val_predict,train_test_split
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

np.random.seed(1)

## Load the data

In [36]:
college = pd.read_csv('C:\\Users\\saads\\Desktop\\Data science\\my project\\mini project 2\\College.csv')
college.rename(columns={'Unnamed: 0':'College'}, inplace=True)
college.head(3)

,College,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54


### Check the names of the variables and the data types contained in the college data

In [37]:
college.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   College      777 non-null    object 
 1   Private      777 non-null    object 
 2   Apps         777 non-null    int64  
 3   Accept       777 non-null    int64  
 4   Enroll       777 non-null    int64  
 5   Top10perc    777 non-null    int64  
 6   Top25perc    777 non-null    int64  
 7   F.Undergrad  777 non-null    int64  
 8   P.Undergrad  777 non-null    int64  
 9   Outstate     777 non-null    int64  
 10  Room.Board   777 non-null    int64  
 11  Books        777 non-null    int64  
 12  Personal     777 non-null    int64  
 13  PhD          777 non-null    int64  
 14  Terminal     777 non-null    int64  
 15  S.F.Ratio    777 non-null    float64
 16  perc.alumni  777 non-null    int64  
 17  Expend       777 non-null    int64  
 18  Grad.Rate    777 non-null    int64  
dtypes: float

### Clean the column names (replace "." by "_" and transform to lower case)

In [38]:
# Clean the column names
college.columns = college.columns.str.replace('.', '_').str.lower()

# Display the cleaned column names
print(college.columns)


Index(['college', 'private', 'apps', 'accept', 'enroll', 'top10perc',
       'top25perc', 'f_undergrad', 'p_undergrad', 'outstate', 'room_board',
       'books', 'personal', 'phd', 'terminal', 's_f_ratio', 'perc_alumni',
       'expend', 'grad_rate'],
      dtype='object')


### Transform the variable "Private" into 1s and 0s rather than yes/no

In [39]:
# Transform the 'private' column to binary values
college['private'] = college['private'].map({'Yes': 1, 'No': 0})

# Display the first few rows to confirm the transformation
college.head()


,college,private,apps,accept,enroll,top10perc,top25perc,f_undergrad,p_undergrad,outstate,room_board,books,personal,phd,terminal,s_f_ratio,perc_alumni,expend,grad_rate
0,Abilene Christian University,1,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,1,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Adrian College,1,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Agnes Scott College,1,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Alaska Pacific University,1,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


### Choose "grad_rate" as target variable

In [40]:
# Define the target variable
y = college['grad_rate']


print(y.head())

0    60
1    56
2    54
3    59
4    15
Name: grad_rate, dtype: int64


### Create a feature matrix containing all variables except "grad_rate" and the college names

In [41]:
# Define the feature variables
X = college.drop(columns=['college', 'grad_rate'])

# Display the first few rows to confirm
print(X.head())

   private  apps  accept  enroll  top10perc  top25perc  f_undergrad  \
0        1  1660    1232     721         23         52         2885   
1        1  2186    1924     512         16         29         2683   
2        1  1428    1097     336         22         50         1036   
3        1   417     349     137         60         89          510   
4        1   193     146      55         16         44          249   

   p_undergrad  outstate  room_board  books  personal  phd  terminal  \
0          537      7440        3300    450      2200   70        78   
1         1227     12280        6450    750      1500   29        30   
2           99     11250        3750    400      1165   53        66   
3           63     12960        5450    450       875   92        97   
4          869      7560        4120    800      1500   76        72   

   s_f_ratio  perc_alumni  expend  
0       18.1           12    7041  
1       12.2           16   10527  
2       12.9           30    873

### Use the standard scaler to rescale your features and response

Hint: to rescale the response variable, you will first have to bring it intro 2D-array form and later to retransform it into 1D-array form.

In [42]:
from sklearn.preprocessing import StandardScaler

In [43]:
# Reshape y into 2D array
y = y.values.reshape(-1, 1)

# Standardize the features and the target variable
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y).ravel()  # Convert back to 1D array after scaling

# Display the first few rows to confirm the transformation
print(X_scaled[:5])
print(y_scaled[:5])


[[ 0.61255305 -0.34688182 -0.32120545 -0.0635089  -0.2585828  -0.19182742
  -0.16811578 -0.20920713 -0.74635589 -0.96490473 -0.60231216  1.27004515
  -0.16302792 -0.1157287   1.01377594 -0.86757419 -0.50191008]
 [ 0.61255305 -0.21088404 -0.03870299 -0.28858421 -0.6556556  -1.3539114
  -0.20978848  0.24430705  0.45749639  1.90920787  1.21587979  0.23551492
  -2.67564554 -3.37817594 -0.4777045  -0.5445722   0.16610985]
 [ 0.61255305 -0.40686563 -0.37631793 -0.47812132 -0.31530749 -0.2928782
  -0.54956538 -0.49709004  0.20130469 -0.55431722 -0.90534415 -0.25958168
  -1.20484498 -0.93134051 -0.30074919  0.58593475 -0.17728996]
 [ 0.61255305 -0.6682606  -0.68168186 -0.69242748  1.84023056  1.67761203
  -0.65807944 -0.52075165  0.62663266  0.99679117 -0.60231216 -0.68817278
   1.18520592  1.17565666 -1.61527433  1.15118822  1.79285145]
 [ 0.61255305 -0.72617601 -0.76455469 -0.78073454 -0.6556556  -0.59603054
  -0.71192387  0.00900549 -0.71650831 -0.21672304  1.51891178  0.23551492
   0.20467

### Fit a 10-fold cross-validated linear regression model for grad_rate using all other features. Evaluate the model performance using cross_val_score, obtain the r2_score and mean_squared_error for each fold and averaged over all folds.

In [15]:
# Initialize the linear regression model
model = LinearRegression()

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=1)

# Obtain cross-validated predictions
y_pred = cross_val_predict(model, X_scaled, y_scaled, cv=kf)

# Compute r2_score and mean_squared_error for each fold
r2_scores = cross_val_score(model, X_scaled, y_scaled, cv=kf, scoring='r2')
mse_scores = -cross_val_score(model, X_scaled, y_scaled, cv=kf, scoring='neg_mean_squared_error')

# Calculate the average r2_score and mean_squared_error
avg_r2_score = np.mean(r2_scores)
avg_mse_score = np.mean(mse_scores)

print(f'R2 Scores for each fold: {r2_scores}')
print(f'Mean R2 Score: {avg_r2_score}')
print(f'Mean Squared Errors for each fold: {mse_scores}')
print(f'Average Mean Squared Error: {avg_mse_score}')


R2 Scores for each fold: [0.57016157 0.35106799 0.38364466 0.28498425 0.2945163  0.45822425
 0.35189315 0.54529099 0.38571707 0.50316098]
Mean R2 Score: 0.4128661220577287
Mean Squared Errors for each fold: [0.44930814 0.64569117 0.49227798 0.68158633 0.74024952 0.52008377
 0.60540212 0.52538338 0.49305118 0.57612769]
Average Mean Squared Error: 0.5729161280620108


### Create a function with the name `predict_from_samples` that will iteratively predict your target from different train-test splits

This will be used to calculate the bias and the variance afterwards.
We want to produce a range of different models fitted on a random choice of data points and making predictions for the remaining data points.


Your function should:

1. take the following arguments:
    * `model`: a regression model 
    * `X`: predictor matrix/dataframe 
    * `y`: target variable 
    * `number_of_splits`: a number indicating how many times the dataset should be split randomly into training and test sets

2. return a dataframe `yhat_tracker` containing columns for 
    * the true values of `y` (obtained from the `y` passed as an argument)
    * the predictions made for y in each of the `number_of_splits` into training/test sets

3. in the function body
    - initialize the dataframe `yhat_tracker` with a single column `y` for the true values
    - initialize a list `rowinds` containing the indices of yhat_tracker
    - create a loop over `number_of_splits`
        - within the loop, create a train/test split of rowinds
        - create training and test sets from y and X by subsetting on the indices obtained from the train/test split
        - fit the model on the training data
        - obtain predictions for those y which are currently in the test set
        - set the predicted values for those `y` which are currently in the training set to `np.nan` 
        - insert the predictions from the current loop as a new column into `yhat_tracker`
    - return yhat_tracker 
    
In the end, the returned data frame should contain one column with the true y-values and `number_of_splits` columns with predicted y-values for the different test sets. Each of the prediction columns contains only as many values as there have been in the test set each time.

In [24]:
def predict_from_samples(model, X, y, number_of_splits=100):
    
    yhat_tracker = pd.DataFrame({'ytrue':y})
    
    rowinds = list(range(X.shape[0]))
    
    for i in range(number_of_splits):
        
        train_inds, test_inds = train_test_split(rowinds, test_size=0.33)
                
        X_train, Y_train = X.iloc[train_inds, :], y[train_inds]
        X_test, Y_test = X.iloc[test_inds, :], y[test_inds]
        
        model.fit(X_train, Y_train)
        yhats = model.predict(X_test)
        
        yhat_tracker['sample'+str(i+1)] = np.nan
        yhat_tracker.iloc[test_inds, -1] = yhats
        
    return yhat_tracker

### Create different predictor datasets

To see what happens to bias and variance as the predictors change, create a few versions of X that have different numbers of predictors in them:

* model $X$ from above including all features
* a model $X_{small}$ including only one feature (e.g. personal)
* a model $X_{overfit}$ created with the patsy formula below 
    - taking the cube takes into account 
        - all original features
        - all features that could be created by squaring or cubing a single column
        - all products of two or three different columns
        - all products of the squared of one column with a different column
        - -1 excludes the intercept)

In [25]:
#overfit_formula = '~ ('+' + '.join(X.columns)+')**3 -1'
#X_overfit = patsy.dmatrix(overfit_formula, data=X, return_type='dataframe')
# Model X_overfit (patsy formula)
overfit_formula = '~ ('+' + '.join(X.columns)+')**3 -1'
X_overfit = patsy.dmatrix(overfit_formula, data=college, return_type='dataframe')

print("Shape of Model X_overfit:", X_overfit.shape)

Shape of Model X_overfit: (777, 833)


In [26]:
#X_small = X[['personal']]
# Model X_small (only one feature)
X_small = college[['personal']]


print("Shape of Model X_small:", X_small.shape)



Shape of Model X_small: (777, 1)


### Use the `predict_from_samples` function you wrote above to get the predicted values for $X$, $X_{small}$, $X_{overfit}$ 

- Run each of your X through the function with the y target vector. 
- Recall that the output of your function has the true values of y in one column and then predicted values of y for the different train-test splits in other columns

In [46]:
import patsy

# Define the patsy formula for X_overfit
overfit_formula = '~ ('+' + '.join(college.columns.drop(['college', 'grad_rate']))+')**3 -1'
X_overfit = patsy.dmatrix(overfit_formula, data=college, return_type='dataframe')

# Initialize a linear regression model
model = LinearRegression()

# Convert y to a 1D array
y_1d = y.ravel()

# Predictions for X
predictions_X = predict_from_samples(model, X, y_1d)

# Predictions for X_small
predictions_X_small = predict_from_samples(model, X_small, y_1d)

# Predictions for X_overfit
predictions_X_overfit = predict_from_samples(model, X_overfit, y_1d)

# Display the predictions
print("Predictions for Model 𝑋:")
print(predictions_X.head())
print("\nPredictions for Model 𝑋𝑠𝑚𝑎𝑙𝑙:")
print(predictions_X_small.head())
print("\nPredictions for Model 𝑋𝑜𝑣𝑒𝑟𝑓𝑖𝑡:")
print(predictions_X_overfit.head())


C:\Users\saads\AppData\Local\Temp\ipykernel_12224\1280828711.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  yhat_tracker['sample'+str(i+1)] = np.nan
C:\Users\saads\AppData\Local\Temp\ipykernel_12224\1280828711.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  yhat_tracker['sample'+str(i+1)] = np.nan


Predictions for Model 𝑋:
   ytrue    sample1    sample2    sample3    sample4  sample5    sample6  \
0     60  56.659838  56.374196        NaN        NaN      NaN  56.352700   
1     56        NaN        NaN        NaN  61.083596      NaN        NaN   
2     54        NaN        NaN        NaN  67.138497      NaN  68.257996   
3     59  77.004029  78.746909        NaN  77.886474      NaN  75.901280   
4     15        NaN  52.850598  53.431149  53.578379      NaN  49.946087   

     sample7    sample8    sample9  ...   sample91   sample92  sample93  \
0        NaN  57.732334  57.042051  ...        NaN  54.125164       NaN   
1        NaN  59.874840  64.396153  ...  64.985297  67.595098       NaN   
2        NaN  66.885025        NaN  ...  67.685618        NaN       NaN   
3        NaN        NaN        NaN  ...        NaN  76.713498       NaN   
4  51.937285        NaN        NaN  ...        NaN        NaN       NaN   

    sample94   sample95   sample96   sample97   sample98   sample99

C:\Users\saads\AppData\Local\Temp\ipykernel_12224\1280828711.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  yhat_tracker['sample'+str(i+1)] = np.nan


In [47]:
def calculate_bias_sq(yhats_df):
    # Take out the true values of y that are in the first column:
    ytrue = yhats_df.iloc[:,0]
    
    # Calculate the mean of the predictions, averaged across the columns.
    # So, all of the predictions for the true y at row 0 would be averaged together
    # and so on for all the rows.
    yhat_means = yhats_df.iloc[:,1:].mean(axis=1)
    
    # Subtract the true value of y from the mean of the predicted values, and square it.
    elementwise_bias_sq = (yhat_means - ytrue)**2
    
    # Take the mean of those squared bias values (across all y)
    mean_bias_sq = np.mean(elementwise_bias_sq)

    return mean_bias_sq

def calculate_variance(yhats_df):
    
    # Calculate the mean of the predicted y's across the columns (mean of yhat for each row)
    yhats_means = yhats_df.iloc[:,1:].mean(axis=1)
    
    # subtract the mean of the yhats from the original yhat values (for each row)
    # and square the result. 
    yhats_devsq = (yhats_df.iloc[:,1:].subtract(yhats_means, axis=0))**2
    
    # Take the mean of the squared deviations from the mean, then 
    # take the mean of those to get the overall variance across the y observations
    yhats_devsq_means = yhats_devsq.mean(axis=1)
    
    return np.mean(yhats_devsq_means)

In [48]:
# Calculate bias and variance for Model 𝑋
bias_X = calculate_bias_sq(predictions_X)
variance_X = calculate_variance(predictions_X)
print("Bias for Model 𝑋:", bias_X)
print("Variance for Model 𝑋:", variance_X)

# Calculate bias and variance for Model 𝑋𝑠𝑚𝑎𝑙𝑙
bias_X_small = calculate_bias_sq(predictions_X_small)
variance_X_small = calculate_variance(predictions_X_small)
print("\nBias for Model 𝑋𝑠𝑚𝑎𝑙𝑙:", bias_X_small)
print("Variance for Model 𝑋𝑠𝑚𝑎𝑙𝑙:", variance_X_small)

# Calculate bias and variance for Model 𝑋𝑜𝑣𝑒𝑟𝑓𝑖𝑡
bias_X_overfit = calculate_bias_sq(predictions_X_overfit)
variance_X_overfit = calculate_variance(predictions_X_overfit)
print("\nBias for Model 𝑋𝑜𝑣𝑒𝑟𝑓𝑖𝑡:", bias_X_overfit)
print("Variance for Model 𝑋𝑜𝑣𝑒𝑟𝑓𝑖𝑡:", variance_X_overfit)

Bias for Model 𝑋: 169.18783944639048
Variance for Model 𝑋: 2.666793738245205

Bias for Model 𝑋𝑠𝑚𝑎𝑙𝑙: 275.3175828355534
Variance for Model 𝑋𝑠𝑚𝑎𝑙𝑙: 0.4733854655338242

Bias for Model 𝑋𝑜𝑣𝑒𝑟𝑓𝑖𝑡: 524158.7662522018
Variance for Model 𝑋𝑜𝑣𝑒𝑟𝑓𝑖𝑡: 4007399.4194041356


### How does regularization affect bias and variance?

Use lasso and/or ridge regression on your dataset with all the predictor variables. In your function `predict_from_samples` replace `model` with your choice.

In [49]:
from sklearn.linear_model import Lasso, Ridge

lasso = Lasso(alpha=2.0)
ridge = Ridge(alpha=2.1)

In [50]:
# Predictions for X using Lasso regression
predictions_X_lasso = predict_from_samples(lasso, X, y_1d)

# Predictions for X using Ridge regression
predictions_X_ridge = predict_from_samples(ridge, X, y_1d)

# Calculate bias and variance for Lasso regression
bias_X_lasso = calculate_bias_sq(predictions_X_lasso)
variance_X_lasso = calculate_variance(predictions_X_lasso)
print("Bias for Model 𝑋 with Lasso regression:", bias_X_lasso)
print("Variance for Model 𝑋 with Lasso regression:", variance_X_lasso)

# Calculate bias and variance for Ridge regression
bias_X_ridge = calculate_bias_sq(predictions_X_ridge)
variance_X_ridge = calculate_variance(predictions_X_ridge)
print("\nBias for Model 𝑋 with Ridge regression:", bias_X_ridge)
print("Variance for Model 𝑋 with Ridge regression:", variance_X_ridge)

C:\Users\saads\AppData\Local\Temp\ipykernel_12224\1280828711.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  yhat_tracker['sample'+str(i+1)] = np.nan


Bias for Model 𝑋 with Lasso regression: 167.7460819799305
Variance for Model 𝑋 with Lasso regression: 2.036087521983139

Bias for Model 𝑋 with Ridge regression: 169.5057353414164
Variance for Model 𝑋 with Ridge regression: 2.7400380353058433


C:\Users\saads\AppData\Local\Temp\ipykernel_12224\1280828711.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  yhat_tracker['sample'+str(i+1)] = np.nan
